In [91]:
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR')   ## set French as local language, useful for date display
# locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # non-windows computer? 

import pandas as pd

In [92]:
raw_data_telegram = pd.read_json('telegram/ChatExport_2020-11-15/result.json')
raw_data_telegram.info()
raw_data_telegram.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      14 non-null     object
 1   type      14 non-null     object
 2   id        14 non-null     int64 
 3   messages  14 non-null     object
dtypes: int64(1), object(3)
memory usage: 576.0+ bytes


,name,type,id,messages
0,Marc,personal_chat,4957211864,"{'id': 591901, 'type': 'service', 'date': '201..."
1,Marc,personal_chat,4957211864,"{'id': 676784, 'type': 'message', 'date': '202..."
2,Marc,personal_chat,4957211864,"{'id': 676785, 'type': 'message', 'date': '202..."
3,Marc,personal_chat,4957211864,"{'id': 676917, 'type': 'message', 'date': '202..."
4,Marc,personal_chat,4957211864,"{'id': 679644, 'type': 'message', 'date': '202..."


In [93]:
def preprocess_telegram_msg(text: str):
    """Process text message for latex, adapt bold, italic, code and underline delimiters"""
    # telegram formating documentation: https://core.telegram.org/api/entities
    regexs = [
        (r"\*\*\*(:?.*?(?<!\*))\*\*\*", r"\\textbf{\\textit{\g<1>}}")  # bold & italic
        ,(r"___(:?.*?(?<!_))___", r"\\textbf{\\textit{\g<1>}}")  # bold & italic
        ,(r"\*\*(:?.*?(?<!\*))\*\*", r"\\textbf{\g<1>}")  # bold
        ,(r"__(:?.*?(?<!_))__", r"\\textbf{\g<1>}")  # bold
        ,(r"<b>(:?.*?)<\\b>", r"\\textbf{\g<1>}")  # bold
        ,(r"<strong>(:?.*?)<\\strong>", r"\\textbf{\g<1>}")  # bold
        ,(r"\*(:?.*?(?<!\*))\*", r"\\textit{\g<1>}")  # italic
        ,(r"_(:?.*?(?<!_))_", r"\\textit{\g<1>}")  # italic
        ,(r"<i>(:?.*?)<\\i>", r"\\textit{\g<1>}")  # italic
        ,(r"<em>(:?.*?)<\\em>", r"\\textit{\g<1>}")  # italic
        ,(r"<u>(:?.*?)<\\u>", r"\\underline{\g<1>}")  # underline
        ,(r"<code>(:?.*?)<\\code>", r"\\texttt{\g<1>}")  # code
    ]  # Beware, regex sustitution order is important

    for regex, subst in regexs:
      text = re.sub(regex, subst, text, 0, re.MULTILINE)
    
    return text

In [102]:
concatenated_table = {'source': [], # telegram or messenger (string)
        'datetime': [], # date (string)
        'sender': [], # M or B (string)
        'message': [], # content in text (string)
        'path': []} # path to images or gifs (list of string)

concatenated_table = pd.DataFrame(concatenated_table)

# Deal with telegram messages
for element in raw_data_telegram["messages"][1:15]:
    
    # simplify sender
    if element["from"] == 'Marc Negre':
        sender = "M"
    elif element["from"] == 'Bénédicte Colnet':
        sender = "B"
    else:
        print("Error on sender on message number "+str(element))
        break
    
    # Deal with picture (apparently several pictures are send as several messages)
    if 'photo' in element:
        photo = element["photo"]
    else:
        photo = []
        
    
    # deal with text that are list or hastag
    if type(element["text"]) == str:
        text = element["text"]
    elif type(element["text"]) == list:
        text = element["text"][0]
        if type(text) == dict:
            text = "\\" + element['text'][0]['text']
    else:
        print("Error in the type of text")
        break
        
    # deal with emoji
    new_text = ""
    for c in text:
        if not(c in emoji.UNICODE_EMOJI):
            new_text = new_text + c
        else:
            new_text = new_text + "\emoji[ios]{"+f"{hex(ord(c))[2:]}"+"}" #Change for f string instead of [2:]
            
    # deal with formatting
    new_text = preprocess_telegram_msg(new_text)
    
    # replace \n with \\
    new_text = new_text.replace("\n", "\\ ")
    
    new_row = {'source' : "Telegram", 'datetime': element["date"], 'sender' : sender, 'message': new_text, 'path': photo}
    concatenated_table = concatenated_table.append(new_row, ignore_index = True)
concatenated_table.shape

(13, 5)

In [103]:
import datetime

# Faced issues with messenger, went on Chatistics repo and parse.py messenger
concatenated_table_messenger = {'source': [], # telegram or messenger (string)
        'datetime': [], # date (string)
        'sender': [], # M or B (string)
        'message': [], # content in text (string)
        'path': []} # path to images or gifs (list of string)

concatenated_table_messenger = pd.DataFrame(concatenated_table_messenger)


with open('messenger/inbox/marcnegre_hwizlpvhxw/message_1.json', encoding='ascii') as f:
    json_data = json.load(f)
participants = json_data["participants"]
#raw_data_messenger = pd.read_json('messenger/inbox/marcnegre_hwizlpvhxw/message_1.json', lines=True)
#raw_data_messenger.info()
#raw_data_messenger.head()
for message in json_data["messages"][1:100]:  # only 100 messages to test
    
    timestamp = message["timestamp_ms"] / 1000
    timestamp = datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S.%f')
    
    sender = message["sender_name"]
    if sender == "BÃ©nÃ©dicte Cnt":
        sender = 'B'
    elif sender == "Marc Negre":
        sender = 'M'
    else:
        print("Error in a sender name")
        print(sender)
        break
    
    # check if 'content' exists in dictionary by checking if get() returned None
    # if not content it is because it is a sticker or picture for example
    if message.get('content') is not None:
        text = message["content"]
        text = text.encode('latin-1').decode('raw_unicode_escape').encode('latin-1').decode('utf-8') #I think better solution exist!
    else:
        text = ""
    
    
    # deal with emoji
    new_text = ""
    for c in text:
        if not(c in emoji.UNICODE_EMOJI):
            new_text = new_text + c
        else:
            new_text = new_text + "\emoji[ios]{"+f"{hex(ord(c))[2:]}"+"}" #Change for f string instead of [2:]
    
    
    
    if message.get('photos') is not None:
        photo = []
        for p in message['photos']:
            photo.append(p['uri']) # now grap the proper png name
            
    else:
        photo = [] # 
    
    new_row = {'source' : "Messenger", 'datetime': timestamp, 'sender' : sender, 'message': new_text, 'path': photo}
    concatenated_table_messenger = concatenated_table_messenger.append(new_row, ignore_index = True)

    
concatenated_table_messenger.shape

(99, 5)

In [104]:
concatenated_table = concatenated_table.append(concatenated_table_messenger)
concatenated_table.shape

(112, 5)

In [114]:
time_col_name = 'datetime'


# format datetime
concatenated_table[time_col_name] = pd.to_datetime(concatenated_table[time_col_name])

# sort by datetime
concatenated_table = concatenated_table.sort_values(time_col_name).reset_index(drop=True)

# add 'date' column
concatenated_table['date'] = concatenated_table[time_col_name].dt.date

# add columns for pretty date and time as strings
concatenated_table['dateStr'] = concatenated_table[time_col_name].apply(lambda x: x.strftime("%A %d %B").replace(' 0', ' ')) # "%A %d %B %Y": +year
concatenated_table['timeStr'] = concatenated_table[time_col_name].apply(lambda x: x.strftime("%H:%M"))


# add boolean colum specifying who is the "right" (as opposed to "left") sender
right_sender = concatenated_table['sender'].unique()[0]  # modify here
concatenated_table['right'] = (concatenated_table['sender'] == right_sender)

# add sender group id column (on a given date, helper column to handle consecutive messages by the same sender)
def f(concatenated_table):
    s = (concatenated_table['right'].shift() != concatenated_table['right'])
    s.iloc[0] = False
    return s.cumsum()

concatenated_table['senderGroup'] = concatenated_table.groupby('date', as_index=False).apply(f).reset_index(drop=True)


concatenated_table

,source,datetime,sender,message,path,date,dateStr,timeStr,right,senderGroup
0,Telegram,2020-06-16 18:01:09.000,B,La pandémie du covid-19 est une maladie nouvel...,[],2020-06-16,Mardi 16 juin,18:01,True,0
1,Telegram,2020-06-16 18:06:17.000,M,Lois mathématiques --,[],2020-06-16,Mardi 16 juin,18:06,False,1
2,Telegram,2020-06-17 09:19:55.000,M,"J'ai lu le teasing d'intro, super bien posé, h...",[],2020-06-17,Mercredi 17 juin,09:19,False,0
3,Telegram,2020-07-07 15:54:05.000,B,Du coup si on résume : \ * \emoji[ios]{1f682} ...,[],2020-07-07,Mardi 7 juillet,15:54,True,0
4,Telegram,2020-07-18 00:16:11.000,B,j'ai meme fait le graphique de la médecin,photos/photo_1@18-07-2020_00-16-11.jpg,2020-07-18,Samedi 18 juillet,00:16,True,0
...,...,...,...,...,...,...,...,...,...,...
107,Messenger,2020-11-13 20:26:08.027,B,\emoji[ios]{1f624},[],2020-11-13,Vendredi 13 novembre,20:26,True,5
108,Messenger,2020-11-13 20:26:17.204,B,Juste savoir quelle affiliation,[],2020-11-13,Vendredi 13 novembre,20:26,True,5
109,Messenger,2020-11-13 20:26:20.306,B,Mais tout est prêt.,[],2020-11-13,Vendredi 13 novembre,20:26,True,5
110,Messenger,2020-11-13 20:26:21.514,B,ET,[],2020-11-13,Vendredi 13 novembre,20:26,True,5


In [115]:
def left_formating(df):
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df).T
    
    table_rows = df.apply(
        lambda x: f'{x.timeStr}&{x.message}\\\\',  # format for left rows
        axis=1
    ).tolist()
    
    text = '\n'.join(
        ['\\noindent', '\\begin{tabular}{p{0.05\\textwidth}p{0.55\\textwidth}}'] +
        table_rows
        + ['\\end{tabular}'])
    
    return text

def left_formating_with_bubbles(df):
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df).T
    
    table_rows = df.apply(
        lambda x: f'{x.message}\\\\',  # format for left rows
        axis=1
    ).tolist()
    
    text = '\n'.join(
        ['\\begin{leftbubbles}'] +
        table_rows
        + ['\\end{leftbubbles}']) 
    return text


def right_formating(df):
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df).T
    
    table_rows = df.apply(
        lambda x: f'{x.message}&{x.timeStr}\\\\' if len(x.message) > 45 
                  else '\multicolumn{1}{r}' + f'{x.message}&{x.timeStr}\\\\',  # format for right rows
        axis=1
    ).tolist()
    
    text = '\n'.join(
        ['\\begin{flushright}', '\\noindent', '\\begin{tabular}{p{0.55\\textwidth}p{0.1\\textwidth}}'] +
        table_rows
        + ['\\end{tabular}', '\\end{flushright}'])

    return text

def right_formating_with_bubbles(df):
    if isinstance(df, pd.Series):
        df = pd.DataFrame(df).T
    
    table_rows = df.apply(
        lambda x: f'{x.message}\\\\',  # format for left rows
        axis=1
    ).tolist()
    
    text = '\n'.join(
        ['\\begin{rightbubbles}'] +
        table_rows
        + ['\\end{rightbubbles}'])
    return text

# By default it will apply bubbles style
def format_msg(df, bubbles = True):
    if bubbles:
        if df['right'].unique()[0]:
            return right_formating_with_bubbles(df)
        return left_formating_with_bubbles(df)
    else:
        if df['right'].unique()[0]:
            return right_formating(df)
        return left_formating(df)

In [116]:
#print(left_formating(df))
#print(right_formating(df))

In [123]:
# Other option to print the tex with one message per bubble.

df = concatenated_table

for date, df_t in df.groupby('date'):
    print(df_t)
    date_str = df_t['dateStr'].unique()[0]
    title = '\\section*{' + date_str + '}'
    print(title)
    

    discussion = '\n'.join(df_t.groupby('timeStr').apply(format_msg))
    
    text = f'{title}\n\n{discussion}'
    
    with open(f'./sections/{date}.tex', 'w', encoding='utf-8') as ft:
        ft.write(text)



     source            datetime sender  \
0  Telegram 2020-06-16 18:01:09      B   
1  Telegram 2020-06-16 18:06:17      M   

                                             message path        date  \
0  La pandémie du covid-19 est une maladie nouvel...   []  2020-06-16   
1                              Lois mathématiques --   []  2020-06-16   

         dateStr timeStr  right  senderGroup  
0  Mardi 16 juin   18:01   True            0  
1  Mardi 16 juin   18:06  False            1  
\section*{Mardi 16 juin}
     source            datetime sender  \
2  Telegram 2020-06-17 09:19:55      M   

                                             message path        date  \
2  J'ai lu le teasing d'intro, super bien posé, h...   []  2020-06-17   

            dateStr timeStr  right  senderGroup  
2  Mercredi 17 juin   09:19  False            0  
\section*{Mercredi 17 juin}
     source            datetime sender  \
3  Telegram 2020-07-07 15:54:05      B   

                                          

In [117]:
# Loop over dates, generate yyyy-mm-dd.tex section files, one per day
# -------------------------------------------------------------------
# Maud version <3

df = concatenated_table


separate_messages = True

for date, df_t in df.groupby('date'):
    date_str = df_t['dateStr'].unique()[0]
    title = '\\section*{' + date_str + '}'
    
    if separate_messages:
        discussion = ' '.join(df_t.groupby('senderGroup').apply(format_msg))
    else:
        discussion = '\n\n'.join(df_t.groupby('senderGroup').apply(format_msg))
    
    text = f'{title}\n\n{discussion}'
    
    with open(f'./sections/{date}.tex', 'w', encoding='utf-8') as ft:
        ft.write(text)